# Dynamic Systems

All physical systems that store and transform energy are all dynamic systems that cannot react instantaneously
to an external solicitation. This because if an energy transformation were to be instantaneous, it would require
an infinite power ($P=dW/dt$), which is impossible to deliver.

These systems are characterized by their capacity to act or not in a stable way, in a fast way, in an oscillating
way, ...

When the underlying physics of dynamic systems is sufficiently known, we have mathematical models capable
of describing their temporal behavior: differential equations. Matlab allows the integration of linear differential
equations with constant coefficients (LTI: Linear Time Invariant), expressed in different forms (and thus their
numerical solutions), but also of non-linear differential equations (for which the most appropriate simulation tool
is Simulink but it will not be discussed here).

As an illustration, let's take the following example: if a force applied to a mass is at the origin of an acceleration,
it is by integrating this force (divided by the mass) that we will obtain the velocity as a function of time and, by
integrating the velocity, we will obtain the distance traveled. So we express the derivatives, Matlab integrates
them. That's easy!

We will continue this tutorial with this well known example: the system mass ($M$), spring ($K$), damper ($R_f$), of
position $y$, subjected to an external force $u$.

![](./12_image1.png)

with $M = 0.5 kg$; $R_f=0.2kg/s$ and $K=1kg/s^2$

The first step in the analysis of a physical system is to write down the laws of physics that govern its operation.

Here, the dynamic equilibrium equation of the forces ($\sum_i F_i = M \cdot a$) on the mass $M$ is writen : $M \frac{d^2y}{dt} + R_f \frac{dy}{dt} + Ky = u$ or in a normalized way : $\frac{d^2y}{dt^2} + \frac{R_f}{M}\frac{dy}{dt} + \frac{K}{M}y = \frac{1}{M}u$

Recall that this differential equation can, **only because it is an LTI equation**, be transformed into a polynomial expression as a function of the variable "$s$" by passing in the Laplace domain.

This passage in the Laplace domain is done with the Laplace transform, and allows us :

- to make our life much easier : we will manipulate algebraic equations rather than differential equations
- to quickly judge the dynamics and stability of the system by identifying its poles (the roots of the
denominator)
- to obtain directly the harmonic response of the system, i.e. in sinusoidal regime after the transient phase,
by replacing "$s$" by "$j\omega$".

Our differential equation becomes, by the Laplace transform : $s^2Y(s) + \frac{R_f}{M}sY(s) + \frac{K}{M}Y(s) = \frac{1}{M}U(s)$

The transfer function is the ratio of the output y to the input u : $$H(s) = \frac{Y(s)}{U(s)} = \frac{\frac{1}{M}}{s^2 + \frac{R_f}{M}s + \frac{K}{M}}$$

This transfer function $H(s)$ will serve as an illustrative example throughout this tutorial. As soon as we talk about the $H(s)$ system, it will be good to remember that the physics behind this transfer function is this mass-spring-damper system.

*NB: If the equation relating to this system were a non-linear differential equation, it would be necessary to work with the tools that are briefly presented at the end of this tutorial, or else it would be necessary to accept to work with small variations of the quantities around an operating point of this system to be able to apply the Laplace transform. This amounts to linearizing this system and allows to use all the theory related to transfer functions. This linearization approach is the one most frequently encountered in industry. Let's remember that the most frequent objective in industry is to control a system with a controller. And it turns out that the LTI tools allowing the implementation of this control are much more understandable and simple to implement than the non linear techniques, which explains why these solutions are preferred.*

## Transfer functions creation (continuous or discrete) in Python

Python knows the "$s$" (for the continuous systems) and "$z$" (for the discrete ones) variables. We can therefore create the transfer function as a fraction of polynomials of "$s$".

But first, the "$s$" variable should be created, which is done with the instruction `tf` from the `control` module:

In [7]:
import control as ct

s=ct.tf('s')

This variable (Laplace variable) can then be manipulated to write the polynomials of the transfer functions. In the case of the above example, by replacing the parameters by their values, this gives :

In [8]:
assert s is not None  # to please the type checker
H = 2/(s**2+0.4*s+2)
H # to make jupyter display the transfer function

TransferFunction(array([2]), array([1. , 0.4, 2. ]))

Note that the instruction `tf` can be used in another way, without creating the variable $s$, by giving it the coefficients of the polynomials in $s$: `tf(num,den)` where `num` is the polynomial of the coefficients in $s$ of the numerator and `den` that of the denominator.

It is also possible to create a transfer function in Python by providing the zeros, the poles and the gain, via the instruction `ct.zpk`.

These ways of doing things being less intuitive, we will prefer the method based on the creation of the variable $s$, and refer you to the `control` module help for the rest.